In [135]:
# Import necessary libraries

import pandas as pd
import nltk
#nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

In [136]:
pfizer_tweets = pd.read_csv('csv/pfizer_processed_tweets.csv', engine='python').drop(['Unnamed: 0'],axis=1)

In [137]:
pfizer_tweets.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,clean_tweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA","Aggregator of Asian American news; scanning diverse sources 24/7/365. RT's, Follows and 'Likes' will fuel me 👩‍💻",2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,['PfizerBioNTech'],Twitter for Android,0,0,False,folk say daikon paste could treat cytokine storm pfizerbiontech
1,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,"['coronavirus', 'SputnikV', 'AstraZeneca', 'PfizerBioNTech', 'Moderna', 'Covid_19']",Twitter for Android,0,0,False,coronavirus sputnikv astrazeneca pfizerbiontech moderna covid russian vaccine create last year
2,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternative news source from citizen journalists that haven't sold out. Real news & real views,2020-04-23 17:58:42,152,580,1473,False,2020-12-12 20:17:19,Explain to me again why we need a vaccine @BorisJohnson @MattHancock #whereareallthesickpeople #PfizerBioNTech… https://t.co/KxbSRoBEHq,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False,explain need vaccine borisjohnson matthancock whereareallthesickpeople pfizerbiontech
3,1337850023531347969,Erin Despas,NaN,"Designing&selling on Teespring. Like 90s Disney tv movies, old school WWE. Dislikes Intolerance, hate, bigots and snakes https://t.co/fa5n4gEHgR",2009-10-30 17:53:54,887,1515,9639,False,2020-12-12 20:01:16,Covid vaccine; You getting it?\n\n #CovidVaccine #covid19 #PfizerBioNTech #Moderna,"['CovidVaccine', 'covid19', 'PfizerBioNTech', 'Moderna']",Twitter Web App,2,1,False,covid vaccine get covidvaccine covid pfizerbiontech moderna
4,1337841934170255365,Tamer Yazar,Turkey-Israel,"Im Market Analyst, also Editor... working (freelance) for some companies that need analysis for their products in Int. Markets \n- Instagram : tamervisits -",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP","['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False,death close mark million people wait pfizerbiontech vaccine first u


In [138]:
pfizer_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22021 entries, 0 to 22020
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                22021 non-null  int64 
 1   user_name         22021 non-null  object
 2   user_location     16535 non-null  object
 3   user_description  20336 non-null  object
 4   user_created      22021 non-null  object
 5   user_followers    22021 non-null  int64 
 6   user_friends      22021 non-null  int64 
 7   user_favourites   22021 non-null  int64 
 8   user_verified     22021 non-null  bool  
 9   date              22021 non-null  object
 10  text              22021 non-null  object
 11  hashtags          20078 non-null  object
 12  source            22019 non-null  object
 13  retweets          22021 non-null  int64 
 14  favorites         22021 non-null  int64 
 15  is_retweet        22021 non-null  bool  
 16  clean_tweet       22021 non-null  object
dtypes: bool(2), 

In [139]:
len(pfizer_tweets)

22021

In [140]:
pfizer_tweets.date = pd.to_datetime(pfizer_tweets.date)
pfizer_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22021 entries, 0 to 22020
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                22021 non-null  int64         
 1   user_name         22021 non-null  object        
 2   user_location     16535 non-null  object        
 3   user_description  20336 non-null  object        
 4   user_created      22021 non-null  object        
 5   user_followers    22021 non-null  int64         
 6   user_friends      22021 non-null  int64         
 7   user_favourites   22021 non-null  int64         
 8   user_verified     22021 non-null  bool          
 9   date              22021 non-null  datetime64[ns]
 10  text              22021 non-null  object        
 11  hashtags          20078 non-null  object        
 12  source            22019 non-null  object        
 13  retweets          22021 non-null  int64         
 14  favorites         2202

In [141]:
pfizer_tweets.isnull().sum()

id                     0
user_name              0
user_location       5486
user_description    1685
user_created           0
user_followers         0
user_friends           0
user_favourites        0
user_verified          0
date                   0
text                   0
hashtags            1943
source                 2
retweets               0
favorites              0
is_retweet             0
clean_tweet            0
dtype: int64

In [142]:
pfizer_tweets = pfizer_tweets.dropna()

In [143]:
len(pfizer_tweets)

14446

In [144]:
pfizer_tweets = pfizer_tweets.drop(['user_name', 'user_location', 'user_description', 'user_created', 'user_followers', 'user_friends', 'user_favourites', 'user_verified', 'hashtags', 'source', 'retweets', 'favorites', 'is_retweet'], axis = 1)

In [145]:
pfizer_tweets

,id,date,text,clean_tweet
0,1340539111971516416,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,folk say daikon paste could treat cytokine storm pfizerbiontech
1,1337858199140118533,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,coronavirus sputnikv astrazeneca pfizerbiontech moderna covid russian vaccine create last year
4,1337841934170255365,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP",death close mark million people wait pfizerbiontech vaccine first u
7,1337789191598575616,2020-12-12 15:59:32,UPDATED: #YellowFever &amp; #COVID19 #ImmunityPassports - Part Two\n\n#SARSCoV2 #PfizerBioNtech #Britain #December… https://t.co/qKT7Rst9aW,updated yellowfever amp covid immunitypassports part two sarscov pfizerbiontech britain december
8,1337783770070409218,2020-12-12 15:38:00,"Coronavirus: Iran reports 8,201 new cases, 221 deaths in the last 24 hours #Iran #coronavirus #PfizerBioNTech… https://t.co/mwDNAdmb7F",coronavirus iran report new case death last hour iran coronavirus pfizerbiontech
...,...,...,...,...
22008,1461536304013856769,2021-11-19 03:26:23,"Several journalists and paid agents of #Pfizer lobbied for Pfizer vaccine in india, they claimed it the best and be… https://t.co/pFaFxyMTbd",several journalist pay agent pfizer lobby pfizer vaccine india claim best
22010,1461507932902756355,2021-11-19 01:33:39,"At this point, Pfizer might just launch subscription services using an IOS app. \n\n#covaxin FTW https://t.co/o8vmQ8pO1U",point pfizer might launch subscription service use ios app covaxin ftw
22018,1463004283209543689,2021-11-23 04:39:37,"@SriLankaTweet @Who If #Pfizer is above all, please tell other companies to stop the production of their Vaccines.… https://t.co/p88L50RfB3",srilankatweet pfizer please tell company stop production vaccine
22019,1462090410193350662,2021-11-20 16:08:12,"You know, just realizing- I know a lot of people who've gotten #Sinovac and #Pfizer, and a few who've gotten #JandJ… https://t.co/8L0QQtDDjE",know realize know lot people get sinovac pfizer gotten jandj


In [146]:
# Use sia.polarity_scores to get sentiment for each tweet

pfizer_tweets['sentiment_1'] = pfizer_tweets.apply(lambda row: sia.polarity_scores(row.clean_tweet)['compound'], axis=1)
pfizer_tweets.head()

,id,date,text,clean_tweet,sentiment_1
0,1340539111971516416,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,folk say daikon paste could treat cytokine storm pfizerbiontech,0.4019
1,1337858199140118533,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,coronavirus sputnikv astrazeneca pfizerbiontech moderna covid russian vaccine create last year,0.2732
4,1337841934170255365,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP",death close mark million people wait pfizerbiontech vaccine first u,-0.5994
7,1337789191598575616,2020-12-12 15:59:32,UPDATED: #YellowFever &amp; #COVID19 #ImmunityPassports - Part Two\n\n#SARSCoV2 #PfizerBioNtech #Britain #December… https://t.co/qKT7Rst9aW,updated yellowfever amp covid immunitypassports part two sarscov pfizerbiontech britain december,0.0000
8,1337783770070409218,2020-12-12 15:38:00,"Coronavirus: Iran reports 8,201 new cases, 221 deaths in the last 24 hours #Iran #coronavirus #PfizerBioNTech… https://t.co/mwDNAdmb7F",coronavirus iran report new case death last hour iran coronavirus pfizerbiontech,-0.5994


In [147]:
#sia.polarity_scores("folk say daikon paste could treat cytokine storm pfizerbiontech")

In [148]:
# Use TextBlob to get the sentiment scores for each tweet

pfizer_tweets['sentiment_2'] = pfizer_tweets.apply(lambda row: TextBlob(row.clean_tweet).sentiment[0], axis=1)
pfizer_tweets.head()

,id,date,text,clean_tweet,sentiment_1,sentiment_2
0,1340539111971516416,2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,folk say daikon paste could treat cytokine storm pfizerbiontech,0.4019,0.000000
1,1337858199140118533,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,coronavirus sputnikv astrazeneca pfizerbiontech moderna covid russian vaccine create last year,0.2732,0.000000
4,1337841934170255365,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP",death close mark million people wait pfizerbiontech vaccine first u,-0.5994,0.250000
7,1337789191598575616,2020-12-12 15:59:32,UPDATED: #YellowFever &amp; #COVID19 #ImmunityPassports - Part Two\n\n#SARSCoV2 #PfizerBioNtech #Britain #December… https://t.co/qKT7Rst9aW,updated yellowfever amp covid immunitypassports part two sarscov pfizerbiontech britain december,0.0000,0.000000
8,1337783770070409218,2020-12-12 15:38:00,"Coronavirus: Iran reports 8,201 new cases, 221 deaths in the last 24 hours #Iran #coronavirus #PfizerBioNTech… https://t.co/mwDNAdmb7F",coronavirus iran report new case death last hour iran coronavirus pfizerbiontech,-0.5994,0.068182


In [149]:

##TextBlob("folk say daikon paste could treat cytokine storm pfizerbiontech").sentiment[0]

In [150]:
# Preview how sentiment scores differ on different tweets

pd.options.display.max_colwidth = 300
r_subset = pfizer_tweets.sample(n=10)
r_subset = r_subset.drop(columns=['id', 'date'])
r_subset.rename(columns={'text':'Initial Tweet'})
r_subset = r_subset.rename(columns={'text': 'Initial Tweet', 'clean_tweet': 'CleanedTweet', 'sentiment_1': 'Sentiment Score Vader', 'sentiment_2':'Sentiment Score TextBlob'})
r_subset.head(10)

,Initial Tweet,CleanedTweet,Sentiment Score Vader,Sentiment Score TextBlob
5691,"many European countries have made dirty deals with #AstraZeneca, #Moderna and #Pfizer to get their poison vaccins o… https://t.co/O0Tl00TRaX",many european country make dirty deal astrazeneca moderna pfizer get poison vaccins,-0.7506,-0.033333
2789,I’ve never been so conflicted in my life 😔 #PfizerBioNTech #immunocompromised,never conflict life pfizerbiontech immunocompromised,0.2411,0.000000
6544,#Moderna and #PfizerBioNTech #vaccines are very effective in real-world conditions at preventing infections”… https://t.co/b7kOOn8jDE,moderna pfizerbiontech vaccine effective real world condition prevent infection,0.4939,0.400000
8503,That second shot hits you like a truckload #PfizerBioNTech #vaccine #Covid,second shot hit like truckload pfizerbiontech vaccine covid,0.3612,0.000000
21741,"#FDA authorizes #Pfizer , #Moderna #boosters for all #adults - #ABC #News https://t.co/hMkXd4htUa",fda authorizes pfizer moderna booster adult abc news,0.0000,0.100000
5992,"@HillaryClinton No not really, that is great production by #Pfizer #Moderna and others. #OperationWarpSpeed… https://t.co/ADLjxrBPPb",hillaryclinton really great production pfizer moderna others operationwarpspeed,0.6590,0.800000
15151,#USDrugMakers #Pfizer and #Moderna have increased the prices of their #mRNA-based #Vaccines against #Covid19 for t… https://t.co/2CNuPYDWWW,usdrugmakers pfizer moderna increase price mrna base vaccine covid,0.3182,-0.800000
16621,@themojostory Did they do the same study for #Pfizer #moderna and #AstraZeneca ? Its just an open statement without a proper comparison,themojostory study pfizer moderna astrazeneca open statement without proper comparison,0.0000,0.000000
21228,Better read these #VaccinationIsNotCompulsary #vaccine #Pfizer #Moderna #AstraZeneca https://t.co/y7P9g3OEZa,well read vaccinationisnotcompulsary vaccine pfizer moderna astrazeneca,0.2732,0.000000
16229,Why not #vaccinate the under 18? Why’s the delay? During the #SecondWave it was planned to get #PfizerVaccine and… https://t.co/DrwUyiBSd9,vaccinate delay secondwave plan get pfizervaccine,-0.3182,0.000000


In [151]:
daily_sentiment = pfizer_tweets[['date', 'sentiment_1', 'sentiment_2']]

In [152]:
len(daily_sentiment)

14446

In [153]:
daily_sentiment.head()

,date,sentiment_1,sentiment_2
0,2020-12-20 06:06:44,0.4019,0.000000
1,2020-12-12 20:33:45,0.2732,0.000000
4,2020-12-12 19:29:07,-0.5994,0.250000
7,2020-12-12 15:59:32,0.0000,0.000000
8,2020-12-12 15:38:00,-0.5994,0.068182


## remove tweets from off-hours

In [154]:
tweets_market_hours = pfizer_tweets.copy()
tweets_market_hours.set_index(pd.to_datetime(tweets_market_hours.date), inplace = True)
tweets_market_hours = tweets_market_hours.drop(columns = ['date'])
#tweets_market_hours = tweets_market_hours.between_time('00:09:30', '16:00:00')
tweets_market_hours.head()

,id,text,clean_tweet,sentiment_1,sentiment_2
date,,,,,
2020-12-20 06:06:44,1340539111971516416,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,folk say daikon paste could treat cytokine storm pfizerbiontech,0.4019,0.000000
2020-12-12 20:33:45,1337858199140118533,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,coronavirus sputnikv astrazeneca pfizerbiontech moderna covid russian vaccine create last year,0.2732,0.000000
2020-12-12 19:29:07,1337841934170255365,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP",death close mark million people wait pfizerbiontech vaccine first u,-0.5994,0.250000
2020-12-12 15:59:32,1337789191598575616,UPDATED: #YellowFever &amp; #COVID19 #ImmunityPassports - Part Two\n\n#SARSCoV2 #PfizerBioNtech #Britain #December… https://t.co/qKT7Rst9aW,updated yellowfever amp covid immunitypassports part two sarscov pfizerbiontech britain december,0.0000,0.000000
2020-12-12 15:38:00,1337783770070409218,"Coronavirus: Iran reports 8,201 new cases, 221 deaths in the last 24 hours #Iran #coronavirus #PfizerBioNTech… https://t.co/mwDNAdmb7F",coronavirus iran report new case death last hour iran coronavirus pfizerbiontech,-0.5994,0.068182


In [155]:
len(pfizer_tweets)

14446

In [156]:
len(tweets_market_hours)

14446

In [157]:
tweets_market_hours = tweets_market_hours.drop(columns = ['id'])

In [158]:
len(tweets_market_hours)

14446

In [159]:
tweets_market_hours.head()

,text,clean_tweet,sentiment_1,sentiment_2
date,,,,
2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,folk say daikon paste could treat cytokine storm pfizerbiontech,0.4019,0.000000
2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,coronavirus sputnikv astrazeneca pfizerbiontech moderna covid russian vaccine create last year,0.2732,0.000000
2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP",death close mark million people wait pfizerbiontech vaccine first u,-0.5994,0.250000
2020-12-12 15:59:32,UPDATED: #YellowFever &amp; #COVID19 #ImmunityPassports - Part Two\n\n#SARSCoV2 #PfizerBioNtech #Britain #December… https://t.co/qKT7Rst9aW,updated yellowfever amp covid immunitypassports part two sarscov pfizerbiontech britain december,0.0000,0.000000
2020-12-12 15:38:00,"Coronavirus: Iran reports 8,201 new cases, 221 deaths in the last 24 hours #Iran #coronavirus #PfizerBioNTech… https://t.co/mwDNAdmb7F",coronavirus iran report new case death last hour iran coronavirus pfizerbiontech,-0.5994,0.068182


In [160]:
resampled_tweets_data = tweets_market_hours.copy()
resampled_tweets_data = resampled_tweets_data.resample('D').mean()
resampled_tweets_data.head()

,sentiment_1,sentiment_2
date,,
2020-12-12,0.034358,0.039161
2020-12-13,0.046367,0.041703
2020-12-14,0.167158,0.121478
2020-12-15,0.176775,0.108430
2020-12-16,0.041658,0.080594


In [161]:
resampled_tweets_data.isnull().sum()

sentiment_1    28
sentiment_2    28
dtype: int64

In [162]:
resampled_tweets_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 347 entries, 2020-12-12 to 2021-11-23
Freq: D
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sentiment_1  319 non-null    float64
 1   sentiment_2  319 non-null    float64
dtypes: float64(2)
memory usage: 8.1 KB


## Remove tweets with sentiment score as 0

In [163]:
tweets_market_hours_sentiment_1 = tweets_market_hours.copy()
tweets_market_hours_sentiment_2 = tweets_market_hours.copy()

In [164]:
(tweets_market_hours_sentiment_1.sentiment_1 == 0).sum() # number of tweets with sentiment score as 0

6690

In [165]:
(tweets_market_hours_sentiment_1.sentiment_2 == 0).sum() # number of tweets with sentiment score as 0

7921

In [166]:
tweets_market_hours_sentiment_1 = tweets_market_hours_sentiment_1[tweets_market_hours_sentiment_1.sentiment_1 != 0]
tweets_market_hours_sentiment_2 = tweets_market_hours_sentiment_2[tweets_market_hours_sentiment_2.sentiment_2 != 0]

In [167]:
(tweets_market_hours_sentiment_1.sentiment_1 == 0).sum() # number of tweets with sentiment score as 0

0

In [168]:
(tweets_market_hours_sentiment_2.sentiment_2 == 0).sum() # number of tweets with sentiment score as 0

0

In [169]:
tweets_market_hours_sentiment_2.head()

,text,clean_tweet,sentiment_1,sentiment_2
date,,,,
2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP",death close mark million people wait pfizerbiontech vaccine first u,-0.5994,0.250000
2020-12-12 15:38:00,"Coronavirus: Iran reports 8,201 new cases, 221 deaths in the last 24 hours #Iran #coronavirus #PfizerBioNTech… https://t.co/mwDNAdmb7F",coronavirus iran report new case death last hour iran coronavirus pfizerbiontech,-0.5994,0.068182
2020-12-12 15:27:42,.@Pfizer will rake in billions from its expensive #CovidVaccine but @AlbertBourla refuses to cut the price for lowe… https://t.co/eC94w3TUl0,pfizer rake billion expensive covidvaccine albertbourla refuse cut price lowe,-0.4215,-0.500000
2020-12-12 14:04:37,"@ZubyMusic 6 deaths so far. \nIt's only death, nothing to worry about then 🤔\n#CovidVaccines\n#Pfizervaccine… https://t.co/HkGTDM5J3f",zubymusic death far death nothing worry covidvaccines pfizervaccine,-0.7502,0.100000
2020-12-14 22:32:45,Members of the @teamsters delivered the first #COVID19Vaccine this morning\n#teamsters\n#UPS\n#COVID19 \n#Pfizer… https://t.co/Ms3N1s9OES,member teamster deliver first covid vaccine morning teamster ups covid pfizer,0.0000,0.250000


In [170]:
tweets_market_hours_sentiment_1 = tweets_market_hours_sentiment_1.resample('D').mean()
tweets_market_hours_sentiment_2 = tweets_market_hours_sentiment_2.resample('D').mean()

In [171]:
tweets_market_hours_sentiment_2.head()

,sentiment_1,sentiment_2
date,,
2020-12-12,-0.108629,0.145455
2020-12-13,0.250307,0.116768
2020-12-14,0.309630,0.246637
2020-12-15,0.355872,0.242884
2020-12-16,0.018411,0.192529


In [172]:
resampled_tweets_data['sentiment_1_no_0'] = tweets_market_hours_sentiment_1.sentiment_1.values
resampled_tweets_data['sentiment_2_no_0'] = tweets_market_hours_sentiment_2.sentiment_2.values

In [173]:
resampled_tweets_data.head()

,sentiment_1,sentiment_2,sentiment_1_no_0,sentiment_2_no_0
date,,,,
2020-12-12,0.034358,0.039161,0.059553,0.145455
2020-12-13,0.046367,0.041703,0.097370,0.116768
2020-12-14,0.167158,0.121478,0.386193,0.246637
2020-12-15,0.176775,0.108430,0.341359,0.242884
2020-12-16,0.041658,0.080594,0.089565,0.192529


In [174]:
resampled_tweets_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 347 entries, 2020-12-12 to 2021-11-23
Freq: D
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sentiment_1       319 non-null    float64
 1   sentiment_2       319 non-null    float64
 2   sentiment_1_no_0  319 non-null    float64
 3   sentiment_2_no_0  318 non-null    float64
dtypes: float64(4)
memory usage: 13.6 KB


In [175]:
daily_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14446 entries, 0 to 22020
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         14446 non-null  datetime64[ns]
 1   sentiment_1  14446 non-null  float64       
 2   sentiment_2  14446 non-null  float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 451.4 KB


In [176]:
tweets_market_hours

,text,clean_tweet,sentiment_1,sentiment_2
date,,,,
2020-12-20 06:06:44,Same folks said daikon paste could treat a cytokine storm #PfizerBioNTech https://t.co/xeHhIMg1kF,folk say daikon paste could treat cytokine storm pfizerbiontech,0.4019,0.000000
2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBioNTech #Moderna #Covid_19 Russian vaccine is created to last 2-4 years… https://t.co/ieYlCKBr8P,coronavirus sputnikv astrazeneca pfizerbiontech moderna covid russian vaccine create last year,0.2732,0.000000
2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mark... millions of people wait #PfizerBioNTech #Vaccine !\n-\nThe first U… https://t.co/uQ3A2f7SVP",death close mark million people wait pfizerbiontech vaccine first u,-0.5994,0.250000
2020-12-12 15:59:32,UPDATED: #YellowFever &amp; #COVID19 #ImmunityPassports - Part Two\n\n#SARSCoV2 #PfizerBioNtech #Britain #December… https://t.co/qKT7Rst9aW,updated yellowfever amp covid immunitypassports part two sarscov pfizerbiontech britain december,0.0000,0.000000
2020-12-12 15:38:00,"Coronavirus: Iran reports 8,201 new cases, 221 deaths in the last 24 hours #Iran #coronavirus #PfizerBioNTech… https://t.co/mwDNAdmb7F",coronavirus iran report new case death last hour iran coronavirus pfizerbiontech,-0.5994,0.068182
...,...,...,...,...
2021-11-19 03:26:23,"Several journalists and paid agents of #Pfizer lobbied for Pfizer vaccine in india, they claimed it the best and be… https://t.co/pFaFxyMTbd",several journalist pay agent pfizer lobby pfizer vaccine india claim best,0.5994,0.500000
2021-11-19 01:33:39,"At this point, Pfizer might just launch subscription services using an IOS app. \n\n#covaxin FTW https://t.co/o8vmQ8pO1U",point pfizer might launch subscription service use ios app covaxin ftw,0.4588,0.000000
2021-11-23 04:39:37,"@SriLankaTweet @Who If #Pfizer is above all, please tell other companies to stop the production of their Vaccines.… https://t.co/p88L50RfB3",srilankatweet pfizer please tell company stop production vaccine,0.0258,0.000000


In [177]:
daily_sentiment = resampled_tweets_data.copy()

In [178]:
daily_sentiment.head()

,sentiment_1,sentiment_2,sentiment_1_no_0,sentiment_2_no_0
date,,,,
2020-12-12,0.034358,0.039161,0.059553,0.145455
2020-12-13,0.046367,0.041703,0.097370,0.116768
2020-12-14,0.167158,0.121478,0.386193,0.246637
2020-12-15,0.176775,0.108430,0.341359,0.242884
2020-12-16,0.041658,0.080594,0.089565,0.192529


In [179]:
len(daily_sentiment)

347

In [180]:
daily_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 347 entries, 2020-12-12 to 2021-11-23
Freq: D
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   sentiment_1       319 non-null    float64
 1   sentiment_2       319 non-null    float64
 2   sentiment_1_no_0  319 non-null    float64
 3   sentiment_2_no_0  318 non-null    float64
dtypes: float64(4)
memory usage: 13.6 KB


In [181]:
daily_sentiment = daily_sentiment.dropna()

In [182]:
daily_sentiment.isnull().sum()

sentiment_1         0
sentiment_2         0
sentiment_1_no_0    0
sentiment_2_no_0    0
dtype: int64

In [183]:
len(daily_sentiment)

318

In [185]:
daily_sentiment.head()

,sentiment_1,sentiment_2,sentiment_1_no_0,sentiment_2_no_0
date,,,,
2020-12-12,0.034358,0.039161,0.059553,0.145455
2020-12-13,0.046367,0.041703,0.097370,0.116768
2020-12-14,0.167158,0.121478,0.386193,0.246637
2020-12-15,0.176775,0.108430,0.341359,0.242884
2020-12-16,0.041658,0.080594,0.089565,0.192529


In [184]:
daily_sentiment.to_csv(r'csv/pfizer_processed_tweets_sentiment_scores.csv', header=True)